# Mathematical Foundations of Computer Graphics and Vision 2022
## EXERCISE 6 - DEEP LEARNING

In [1]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.io import read_image
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
from pytorch_msssim import ssim
from torchvision.transforms import InterpolationMode


import matplotlib.pyplot as plt
import os
import numpy as np
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


## 1.1. Task 1 - Datasets, Preprocessing and Data loading

In [3]:
class SRDataset(Dataset):
    """Define training/valid dataset loading methods.
    Args:
        image_dir (str): Train/Valid dataset address.
        image_size (int): High resolution image size.
        upscale_factor (int): Image up scale factor.
        mode (str): Data set loading method, the training data set is for data enhancement, and the
            verification dataset is not for data enhancement.
    """

    def __init__(self, image_dir, image_size = 64, upscale_factor = 2, jitter_val = 0.2, mode = 'Train', interpolation = InterpolationMode.BILINEAR) -> None:
        super(SRDataset, self).__init__()
        self.image_file_names = [os.path.join(image_dir, image_file_name) for image_file_name in os.listdir(image_dir)]
        self.image_size = image_size
        self.upscale_factor = upscale_factor
        self.jitter_val = jitter_val
        self.mode = mode
        self.interpolation = interpolation

    def __getitem__(self, index):
        image = read_image(self.image_file_names[index]).float()/255.
        if self.mode == 'Train':
            hr_transformer = transforms.Compose([
                transforms.RandomCrop(self.image_size),
                transforms.ColorJitter(brightness=self.jitter_val, contrast=self.jitter_val, saturation=self.jitter_val, hue=self.jitter_val),
            ])  
        else:
            hr_transformer = transforms.Compose([
                transforms.CenterCrop(self.image_size),
                transforms.ColorJitter(brightness=self.jitter_val, contrast=self.jitter_val, saturation=self.jitter_val, hue=self.jitter_val),
            ])
        
        lr_transformer = transforms.Compose([
            transforms.Resize(size=(int(self.image_size / self.upscale_factor), int(self.image_size / self.upscale_factor)), interpolation = self.interpolation),
        ])
        hr_image = hr_transformer(image)
        lr_image = lr_transformer(hr_image)
        return lr_image, hr_image

    def __len__(self):
        return len(self.image_file_names)

In [4]:
train_path = './train'
test_path = './eval'
train_dataset = SRDataset(train_path)
test_dataset = SRDataset(test_path)
test_dataset_cubic = SRDataset(test_path, interpolation = InterpolationMode.BICUBIC)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=0 if device == 'cpu' else 2,
    drop_last=True,
    pin_memory=True,
)
eval_dataloader = DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=0 if device == 'cpu' else 2,
    drop_last=True,
    pin_memory=True,
)
eval_dataloader_cubic = DataLoader(
    test_dataset_cubic,
    batch_size=4,
    shuffle=True,
    num_workers=0 if device == 'cpu' else 2,
    drop_last=True,
    pin_memory=True,
)

print(f" * Dataset contains {len(train_dataset)} image(s).")
for _, batch in enumerate(train_dataloader, 0):
    lr_image, hr_image = batch
    torchvision.io.write_png(lr_image[0, ...].mul(255).byte(), "lr_image.png")
    torchvision.io.write_png(hr_image[0, ...].mul(255).byte(), "hr_image.png")
    break # we deliberately break after one batch as this is just a test

 * Dataset contains 301 image(s).


## 1.2. Task 2 - Create the Model

In [5]:
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models

In [6]:
class BasicSRModel(nn.Module):
    def __init__(self, upscale_factor = 2, layers = 10, residual = False) :
        super(BasicSRModel, self).__init__()
        self.up_sample = nn.Upsample(scale_factor=upscale_factor, mode='bilinear')
        self.conv_first = nn.Conv2d(3, 64, (3, 3), (1, 1), (1, 1))
        self.conv_last = nn.Conv2d(64, 3, (3, 3), (1, 1), (1, 1))
        modules = []
        for i in range(layers):
            modules.append(nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1)))
            modules.append(nn.LeakyReLU())
        self.conv_middle = nn.Sequential(*modules)
        self.residual = residual

    def forward(self, x):
        x_up = self.up_sample(x)
        out1 = self.conv_first(x_up)
        out2 = self.conv_middle(out1)
        out = self.conv_last(out2)
        if self.residual:
            out += x_up
        return out

In [7]:
# learning_rate [1e-2, 1e-3, 1e-5, 1e-6]
learning_rate = 1e-6
save_iterval = 200
number_of_epochs = 1000
save_dir = './models'

model = BasicSRModel(residual = True).to(device)
loss_function = nn.L1Loss().to(device)
mse = nn.MSELoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),
                                 lr=learning_rate)
num_params = 0
for param in model.parameters():
    num_params += param.numel()
print(num_params)

372803


## 1.3. Task 3 - Implement the Training Loop

In [8]:
# Writer will output to ./runs/ directory by default
writer = SummaryWriter()

In [ ]:
# average loss or use global id
for epoch in tqdm(range(number_of_epochs)):
    cnt = 0
    l1 = 0
    psnr = 0
    SSIM = 0
    min_loss = -1
    for batch_id, batch in enumerate(train_dataloader):
#         global_id = len(train_dataloader) * epoch + batch_id
        low_res, high_res = batch
        low_res = low_res.to(device)
        high_res = high_res.to(device)
        optimizer.zero_grad()
        high_res_prediction = model(low_res)
        loss = loss_function(high_res_prediction, high_res)
        
        l1 += loss
        psnr += -10*torch.log10(mse(high_res_prediction, high_res))
        SSIM += ssim( high_res_prediction, high_res, data_range=255, size_average=True)
        cnt += 1
        
        loss.backward()
        optimizer.step()
        
    if min_loss == -1 or min_loss > (float(l1) / float(cnt)):
        min_loss = float(l1) / float(cnt)
        torch.save(model.state_dict(), '{:s}/model_best_lr_{:.0E}.pth'.format(save_dir, learning_rate))
        
    writer.add_scalar('L1/train', float(l1) / float(cnt), epoch)
    writer.add_scalar('PSNR/train', float(psnr) / float(cnt), epoch)
    writer.add_scalar('SSIM/train', float(SSIM) / float(cnt), epoch)
    
    cnt = 0
    l1 = 0
    psnr = 0
    SSIM = 0
    for batch_id, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            low_res, high_res = batch
            low_res = low_res.to(device)
            high_res = high_res.to(device)
            high_res_prediction = model(low_res)
            loss = loss_function(high_res_prediction, high_res)

            l1 += loss
            psnr += -10*torch.log10(mse(high_res_prediction, high_res))
            SSIM += ssim( high_res_prediction, high_res, data_range=255, size_average=True)
            cnt += 1
    writer.add_scalar('L1/evaluation', float(l1) / float(cnt), epoch)
    writer.add_scalar('PSNR/evaluation', float(psnr) / float(cnt), epoch)
    writer.add_scalar('SSIM/evaluation', float(SSIM) / float(cnt), epoch)
    if (epoch + 1) % save_iterval == 0 or (epoch + 1) == number_of_epochs:    
        torch.save(model.state_dict(), '{:s}/model_iter_{:d}_lr_{:.0E}.pth'.format(save_dir, epoch, learning_rate))

  0%|                                                                                      | 0/1000 [00:00<?, ?it/s][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
  0%|                                                                            | 1/1000 [00:08<2:24:45,  8.69s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthre

  2%|█▏                                                                         | 16/1000 [02:20<2:23:50,  8.77s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
  2%|█▎                                                                         | 17/1000 [02:29<2:24:27,  8.82s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthre

  3%|██▍                                                                        | 32/1000 [04:38<2:19:04,  8.62s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
  3%|██▍                                                                        | 33/1000 [04:47<2:18:14,  8.58s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthre

  5%|███▌                                                                       | 48/1000 [06:51<2:12:29,  8.35s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
  5%|███▋                                                                       | 49/1000 [07:00<2:14:54,  8.51s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthre

  6%|████▊                                                                      | 64/1000 [09:05<2:10:15,  8.35s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
  6%|████▉                                                                      | 65/1000 [09:14<2:10:22,  8.37s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthre

...

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
 61%|█████████████████████████████████████████████▏                            | 611/1000 [1:24:20<51:10,  7.89s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
 61%|█████████████████████████████████████████████▎                            | 612/1000 [1:24:28<51:12,  7.92s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthre

 63%|██████████████████████████████████████████████▍                           | 627/1000 [1:26:27<48:42,  7.84s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
 63%|██████████████████████████████████████████████▍                           | 628/1000 [1:26:35<49:15,  7.95s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthre

 64%|███████████████████████████████████████████████▌                          | 643/1000 [1:28:29<38:06,  6.40s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
 64%|███████████████████████████████████████████████▋                          | 644/1000 [1:28:34<36:35,  6.17s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthre

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
 66%|████████████████████████████████████████████████▊                         | 659/1000 [1:30:00<34:07,  6.00s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
 66%|████████████████████████████████████████████████▊                         | 660/1000 [1:30:07<34:55,  6.16s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthre

[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
 68%|█████████████████████████████████████████████████▉                        | 675/1000 [1:31:33<31:32,  5.82s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
 68%|██████████████████████████████████████████████████                        | 676/1000 [1:31:38<31:00,  5.74s/it][W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthreadpool)
[W pthreadpool-cpp.cc:90] Warning: Leaking Caffe2 thread-pool after fork. (function pthre

In [ ]:
model = BasicSRModel()
model.load_state_dict(torch.load('{:s}/model_best_lr_{:.0E}.pth'.format(save_dir, learning_rate)))

## 1.4. Task 4 - Model Evaluation

In [ ]:
cnt = 0
psnr = 0
SSIM = 0
for batch_id, batch in enumerate(eval_dataloader):
    with torch.no_grad():
        low_res, high_res = batch
        low_res = low_res.to(device)
        high_res = high_res.to(device)
        high_res_prediction = model(low_res)
        psnr += -10*torch.log10(mse(high_res_prediction, high_res))
        SSIM += ssim( high_res_prediction, high_res, data_range=255, size_average=True)
        cnt += 1
print("Evaluation PSNR (bilinear): {:f}".format(float(psnr) / float(cnt)))
print("Evaluation SSIM (bilinear): {:f}".format(float(SSIM) / float(cnt)))

In [ ]:
cnt = 0
psnr = 0
SSIM = 0
for batch_id, batch in enumerate(eval_dataloader_cubic):
    with torch.no_grad():
        low_res, high_res = batch
        low_res = low_res.to(device)
        high_res = high_res.to(device)
        high_res_prediction = model(low_res)
        psnr += -10*torch.log10(mse(high_res_prediction, high_res))
        SSIM += ssim( high_res_prediction, high_res, data_range=255, size_average=True)
        cnt += 1
print("Evaluation PSNR (bicubic): {:f}".format(float(psnr) / float(cnt)))
print("Evaluation SSIM (bicubic): {:f}".format(float(SSIM) / float(cnt)))